In [ ]:
# Population density --> Covid incidence ?

In [ ]:
import sys
import numpy, matplotlib, pandas, wget # pip3 install numpy, matplotlib, pandas, wget

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pandas.set_option('display.max_columns', None)

In [ ]:
OPENDATASOFT_URL01 = "https://public.opendatasoft.com/explore/dataset/covid-19-germany-landkreise/table"
OPENDATASOFT_URL02 = "https://public.opendatasoft.com/explore/dataset/covid-19-germany-landkreise/download/?format=csv&lang=en&use_labels_for_header=true&csv_separator=%3B"
FILENAME = "covid-19-germany-landkreise.csv"

def download_kreise(url1=OPENDATASOFT_URL01, url2=OPENDATASOFT_URL02, out=FILENAME):
    print ("Downloading large table with Kreise information. For infos see")
    print (url1)
    print ("Patience please ...", end=" ")
    filename = wget.download(url2, out=out)
    print ("Done -->", filename)
    return filename

filename = download_kreise(url1=OPENDATASOFT_URL01, url2=OPENDATASOFT_URL02, out=FILENAME)
filename

In [ ]:
LKG=pandas.read_csv(filename, sep=';')
LKG.columns

In [ ]:
LKG

In [ ]:
LKG.describe()

In [ ]:
# print("\n".join(map(str, sorted(list(LKG['Regional code'])))))

In [ ]:
uninteresting=['Administrative level', 'Specific domains', 'Regional code of the local authority','Type of division', 'Type of division code', 'Name construction indication', 'Regierungsbezirk code','recovered','RS_0', 'Land factor', 'WSK']
kreise=LKG.drop(uninteresting, axis=1)

In [ ]:
# Brandenburg and Berlin
selection=(10999<LKG['Regional code']) & (13000>LKG['Regional code'])
kreise[selection]
kreise[["Regional code", "Short name", "Cadastral area", "SHAPE_Area"]][selection]

In [ ]:
print('fix missing "Cadastral area" data for Berlin, by extrapolating from given "SHAPE_Area"')
print()

print("Explore approximate proportionality:")
print((kreise['Cadastral area']/kreise['SHAPE_Area']).describe())
M=(kreise['Cadastral area']/kreise['SHAPE_Area']).mean()
print('factor shape-->cadastral:', M)
print()

print("Now apply that to Berlin:")

berlin=(10999<LKG['Regional code']) & (11013>LKG['Regional code'])

areatotal=0
for k in kreise[berlin].index:
    SA=kreise['SHAPE_Area'][k]
    CA_extrapolated = SA*M
    print(k, SA, CA_extrapolated)
    kreise.at[k, 'Cadastral area'] = CA_extrapolated
    areatotal+=CA_extrapolated

print()
print("Berlin area scaled and summed: %.1f" % areatotal)
print("Wikipedia: Area City/State     891.1 km2 ")
print ("approximation is ... good enough")

In [ ]:
kreise[["Short name", "Cadastral area", "SHAPE_Area"]][selection]

In [ ]:
kreise['Population density']=kreise['Population']/kreise['Cadastral area']
someColumns1=["Short name", "Population density", "Population", "Cadastral area", 'Cases per 100k persons in the last 7 days']
popDensiSorted=kreise[someColumns1].sort_values("Population density", ascending=False).reset_index().drop(['index'], axis=1)
popDensiSorted

In [ ]:
popDensiSorted.head(15)

In [ ]:
ax=popDensiSorted.head(30).reset_index().plot(x='Short name', y='Population density', kind='bar', logy=True, rot=90, figsize=(20,15), title="Population divided by km² Area of 'Kreis' (district) for the top most dense in Germany. Beware that y-axis is logarithmic.")
matplotlib.pyplot.tight_layout()
ax.figure.savefig(fname="img/populationDensity_top30.png")


In [ ]:
ax=popDensiSorted.head(100).plot(x='Short name', y='Population density', kind='bar', logy=True, rot=90, figsize=(25,15), title="Population divided by km² Area of 'Kreis' (district), for top100 most dense in Germany. Beware that y-axis is logarithmic.")
matplotlib.pyplot.tight_layout()
ax.figure.savefig(fname="img/populationDensity_top100.png")

In [ ]:
ax=popDensiSorted.plot(x='Short name', y='Population density', kind='scatter', logy=True, rot=90, figsize=(25,15), title="Population divided by km² Area of 'Kreis' (district), for all of Germany. Beware: y-axis is logarithmic. x-labels only every 4th.")
ax.xaxis.set_major_locator(matplotlib.pyplot.MaxNLocator(100))
matplotlib.pyplot.tight_layout()
ax.figure.savefig(fname="img/populationDensity_all.png")

In [ ]:
maxIncidence=max(popDensiSorted['Cases per 100k persons in the last 7 days'])
minIncidence=min(popDensiSorted['Cases per 100k persons in the last 7 days'])
minIncidence, maxIncidence


In [ ]:
ax=popDensiSorted.plot(x='Short name', y='Cases per 100k persons in the last 7 days', kind='scatter', logy=True, rot=90, figsize=(25,15), title="(Oct 15th 2020) 'Cases per 100k persons in the last 7 days', with all German districts ('Kreis') SORTED by DECREASING population density. Beware: y-axis is logarithmic. x-labels only every 4th.")
ax.set_ylim([1, maxIncidence*1.10])
ax.xaxis.set_major_locator(matplotlib.pyplot.MaxNLocator(100))
matplotlib.pyplot.tight_layout()
ax.figure.savefig(fname="img/populationDensity-sorted-districts_vs_incidence.png")

In [ ]:
ax=popDensiSorted.plot(x='Population density', y='Cases per 100k persons in the last 7 days', kind='scatter', logx=True, logy=True, rot=90, figsize=(15,10), title="(Oct 15th 2020) 'Cases per 100k persons in the last 7 days' versus 'Population density', for each German district ('Kreis'). Beware: both axes logarithmic.")
ax.set_ylim([1, maxIncidence*1.10])
matplotlib.pyplot.tight_layout()
ax.figure.savefig(fname="img/populationDensity_vs_incidence_data_loglog.png")

In [ ]:
clean7=kreise[someColumns1][kreise['Cases per 100k persons in the last 7 days']==0]
if not clean7.empty:
    
    print ("kick out zero incidence regions")
    print (clean7)
    kreise=kreise.drop(clean7.index)
else:
    print ("No district had zero 7-days-incidence.")

In [ ]:
# regression

x=numpy.log(kreise['Population density'])
y=numpy.log(kreise['Cases per 100k persons in the last 7 days'])

coeff = numpy.polyfit(x=x, y=y, deg=1)
print(coeff)
poly1d_fn = numpy.poly1d(coeff) 
poly1d_fn

weights=kreise['Population'] # make districts count more which have larger population
coeff2 = numpy.polyfit(x=x, y=y, deg=1, w=weights)
print(coeff2)
poly1d_fn_2 = numpy.poly1d(coeff2) 

coeff3 = numpy.polyfit(x=x, y=y, deg=3, w=weights)
print(coeff3)
poly1d_fn_3 = numpy.poly1d(coeff3) 

In [ ]:
# log log plot and regression fits

plt=matplotlib.pyplot
fig=plt.figure(figsize=(15,15))
gs = fig.add_gridspec(1, 1)
ax1 = fig.add_subplot(gs[0, 0])

norm=matplotlib.colors.LogNorm()
cmap='cool'
# cmap=matplotlib.cm.coolwarm

cax1=ax1.scatter(x,y,
                 c=weights, cmap=cmap, norm=norm,
                 marker='o', label="All German districts ('Kreise'): Recent incidence versus population density")
fig.colorbar(cax1, label='Population', orientation='horizontal')

ax1.plot(x, poly1d_fn(x), 'b--', label="simple regression through all these points --> coefficients=(%.4f,%.4f)" % (coeff[0],coeff[1]))
ax1.plot(x, poly1d_fn_2(x), 'g--', label="population-weighted regression ('large count more'), coeff=(%.4f,%.4f)" % (coeff2[0],coeff2[1]))
ax1.plot(sorted(x), sorted(poly1d_fn_3(x)), 'r--', label="population-weighted regression with deg=3, coeff=(%.4f,%.4f,%.4f,%.4f)" % (coeff3[0],coeff3[1],coeff3[2],coeff3[3]))

ax1.set_xlabel("LOG (Population density)")
ax1.set_ylabel("LOG (Cases per 100k persons in the last 7 days)")

ax1.set_title("(Oct 15th 2020) LOG LOG plot of 'Incidence' versus 'Population density' and simple regression fits")
L=ax1.legend(loc=4)

matplotlib.pyplot.tight_layout()
ax1.figure.savefig(fname="img/populationDensity_vs_incidence_data-and-fits_loglog.png")

In [ ]:
plt=matplotlib.pyplot
fig=plt.figure(figsize=(15,15))
gs = fig.add_gridspec(1, 1)
ax1 = fig.add_subplot(gs[0, 0])

norm=matplotlib.colors.LogNorm()
cmap='cool'
# cmap=matplotlib.cm.coolwarm

cax1=ax1.scatter(x=kreise['Population density'],y=kreise['Cases per 100k persons in the last 7 days'],
                 c=weights, cmap=cmap, norm=norm,
                 marker='o', label="All German districts ('Kreise'): Recent incidence versus population density")
fig.colorbar(cax1, label='Population', orientation='horizontal')

# to plot these with connecting lines, they must be in order:
fit_x, fit_y = sorted(kreise['Population density']), sorted(numpy.exp(poly1d_fn_2(x)).tolist())
fit3_x, fit3_y = sorted(kreise['Population density']), sorted(numpy.exp(poly1d_fn_3(x)).tolist())

ax1.plot(fit_x, fit_y, 'g--', label="population-weighted fit ('large count more') of loglog data, coeff=(%.4f,%.4f)" % (coeff2[0],coeff2[1]))
ax1.plot(fit3_x, fit3_y, 'r--', label="population-weighted fit of loglog data with deg=3, coeff=(%.4f,%.4f,%.4f,%.4f)" % (coeff3[0],coeff3[1],coeff3[2],coeff3[3]))

ax1.set_xlabel("Population density: People per square km")
ax1.set_ylabel("Cases per 100k persons in the last 7 days")

ax1.set_title("(Oct 15th 2020) LIN LIN plot of 'Incidence' versus 'Population density' and simple regression fits")
L=ax1.legend(loc=4)

# showOnlySmallPopulationDensities=ax1.set_xlim([0,2000])

matplotlib.pyplot.tight_layout()
ax1.figure.savefig(fname="img/populationDensity_vs_incidence_data-and-fits_linlin.png")